## 登录admin

In [140]:
# 获取token
import requests

login_url = "https://admin.summerfarm.net/authentication/auth/username/login"
login_data = {
    "username": "peng.tang@summerfarm.net",
    "password": "summer@202302",
}

token = requests.post(login_url, data=login_data).json()

print(token)


headers = {
    'token': token.get("data").get("token"),
    'xm-rqid': 'create_fake_merchant_tp',
    'xm-uid': '2047',
    'Content-Type': 'application/json;charset=UTF-8',
}

print(headers)

{'code': 'SUCCESS', 'data': {'authId': 2082, 'bizUserId': 2047, 'phone': '18618107293', 'realname': '唐鹏', 'token': 'admin__6852aa40-3865-495e-a06c-0777b5ffac21', 'userBaseId': 2080}, 'msg': '', 'success': True}
{'token': 'admin__6852aa40-3865-495e-a06c-0777b5ffac21', 'xm-rqid': 'create_fake_merchant_tp', 'xm-uid': '2047', 'Content-Type': 'application/json;charset=UTF-8'}


In [141]:
def get_product_info(pd_id=11087):
    url = "https://admin.summerfarm.net/sf-mall-manage/product/query/info"
    data = {"pdId": pd_id}
    return requests.post(url=url, headers=headers, json=data).json().get("data")


# Define the 'CASE WHEN' logic
def open_sale_case_when(open_sale=-1):
    if open_sale == 0:
        return "上架"
    elif open_sale == 1:
        return "有库存时上架"
    elif open_sale == 2:
        return "定时上架"
    elif open_sale == 3:
        return "有库存时上架(永久生效)"
    else:
        return "未定义"
    
pd_list = (
    requests.get(
        "https://admin.summerfarm.net/sf-mall-manage/product/query/selectPage?grandCategoryId=1132&pageIndex=1&pageSize=500",
        headers=headers,
    )
    .json()
    .get("data")
    .get("list")
)

## 构建鲜沐的类目树

In [142]:
url = "https://admin.summerfarm.net/category"
all_xianmu_category = requests.get(url, headers=headers).json().get("data")
all_xianmu_category_pop = []
for cate in all_xianmu_category:
    if "POP" in cate["category"]:
        all_xianmu_category_pop = cate.get("categoryList")
        break
# print(all_xianmu_category_pop)
all_xianmu_category_pop_map = {}
for second_level in all_xianmu_category_pop:
    second_category_name = second_level.get("category", "")
    for leaf_level in second_level.get("categoryList", []):
        all_xianmu_category_pop_map[f"{leaf_level.get('id','haha')}"] = (
            f"{second_category_name}/{leaf_level['category']}"
        )

all_xianmu_category_pop_map

{'1167': '榴莲/金枕榴莲',
 '1168': '榴莲/干尧榴莲',
 '1169': '榴莲/其他榴莲',
 '1267': '菠萝蜜/红肉菠萝蜜',
 '1268': '菠萝蜜/其他菠萝蜜',
 '1269': '草莓/红颜',
 '1270': '草莓/蒙特瑞',
 '1271': '草莓/其他草莓',
 '1244': '车厘子/樱桃/国产车厘子',
 '1245': '车厘子/樱桃/进口车厘子',
 '1246': '车厘子/樱桃/樱桃',
 '1247': '车厘子/樱桃/其他车厘子/樱桃',
 '1261': '凤梨/菠萝/金钻凤梨',
 '1262': '凤梨/菠萝/进口凤梨',
 '1263': '凤梨/菠萝/普通凤梨',
 '1264': '凤梨/菠萝/菠萝',
 '1265': '凤梨/菠萝/其他凤梨/菠萝',
 '1274': '柑/橙/沃柑',
 '1275': '柑/橙/其他柑',
 '1276': '柑/橙/伦晚橙',
 '1277': '柑/橙/夏橙',
 '1278': '柑/橙/脐橙',
 '1279': '柑/橙/其他橙',
 '1248': '火龙果/进口红心',
 '1249': '火龙果/进口白心',
 '1250': '火龙果/国产红心',
 '1251': '火龙果/其他火龙果',
 '1286': '橘/桔/金桔',
 '1287': '橘/桔/蜜桔',
 '1288': '橘/桔/其他橘/桔',
 '1220': '蓝莓/国产蓝莓',
 '1221': '蓝莓/进口蓝莓',
 '1222': '蓝莓/其他蓝莓',
 '1197': '梨/香梨',
 '1198': '梨/丰水梨',
 '1199': '梨/皇冠梨',
 '1200': '梨/雪花梨',
 '1201': '梨/贡梨',
 '1202': '梨/红香酥梨',
 '1203': '梨/其他梨',
 '1212': '李子/青脆李',
 '1213': '李子/蜂糖李',
 '1214': '李子/五月脆李',
 '1215': '李子/半边红李',
 '1216': '李子/其他李子',
 '1232': '荔枝/桂味',
 '1233': '荔枝/妃子笑',
 '1234': '荔枝/红花',
 '1235': '荔枝/兰竹',
 '123

## 从鲜沐admin后台获取所有的POP商品

In [143]:
import json
import pandas as pd


def extract_product_fields(json_data):
    # Initialize a list to hold the extracted data
    extracted_data = []
    keyValueList = ",".join(
        [
            f"{kv['name']}:{kv['productsPropertyValue']}"
            for kv in json_data.get("keyValueList", [])
        ]
    )

    # Loop through each item in the inventory detail list
    for item in json_data.get("inventoryDetailVOS", []):
        # Concatenate saleValueList
        sale_value_list = item.get("saleValueList", [])
        buyer_name = item.get("buyerName", "")
        sale_value_string = ",".join(
            f"{sv['name']}:{sv['productsPropertyValue']}" for sv in sale_value_list
        )
        # Loop through each area SKU in the item
        for area_sku in item.get("areaSkuVOS", []):
            # Extract the necessary fields and create a dictionary
            extracted_dict = {
                "sku": area_sku.get("sku", ""),
                "categoryName": all_xianmu_category_pop_map.get(
                    f'{json_data.get("categoryId")}', "哈哈哈哈"
                ),
                "openSale": open_sale_case_when(area_sku.get("openSale", -1)),
                "pdId": item.get("pdId", ""),
                "sku_name": json_data.get("productName", ""),
                "pd_name": json_data.get("pdName", ""),
                "realName": item.get("realName", ""),
                "picture": "https://azure.summerfarm.net/"
                + json_data.get("picturePath", "404.jpg"),
                "price": area_sku.get("price", 0),
                "supplyPrice": json.loads(item.get("createRemark", "{}")).get(
                    "supplyPrice", 0
                ),
                "areaNo": area_sku.get("areaNo", ""),
                "areaName": area_sku.get("areaName", ""),
                "netWeightNum": item.get("netWeightNum", 0),
                "weightNum": item.get("weightNum", 0),
                "weight": item.get("weight", ""),
                "sku_spec": sale_value_string,
                "spu_spec": keyValueList,
                "buyer_name": buyer_name,
            }
            # Add the dictionary to the list
            extracted_data.append(extracted_dict)

    return extracted_data

In [144]:
product_list = []

for product in pd_list:
    product = get_product_info(product["pdId"])
    product_list.append(product)

In [145]:
from datetime import datetime, timedelta

sku_list = []

for product in product_list:
    extracted_data = extract_product_fields(product)
    sku_list.extend(extracted_data)

sku_list_df = pd.DataFrame(sku_list)
sku_list_df.to_csv(
    f"./data/鲜果POP全部SKU_{datetime.now().strftime('%Y%m%d')}.csv", index=False
)
sku_list_df.sort_values(by=["categoryName", "pd_name"], inplace=True)
sku_list_df.head(2)

,sku,categoryName,openSale,pdId,sku_name,pd_name,realName,picture,price,supplyPrice,areaNo,areaName,netWeightNum,weightNum,weight,sku_spec,spu_spec,buyer_name
41,1296235406315,其他鲜果/释迦/莲雾,上架,11153,海南玫瑰释迦12头净重3.8斤起A级,海南玫瑰释迦12头净重3.8斤起A级,海南玫瑰释迦12头净重3.8斤起A级,https://azure.summerfarm.net/picture-path/hbw9...,999.0,NaN,44240,POP杭州鲜果,1.9,2.5,净重3.8-4.2斤/A级/12头,"规格:净重3.8-4.2斤,果规:12头,级别:A级","品种:玫瑰,产地:海南,储藏温度:5-15度",小明
184,1298087783322,其他鲜果/雪莲果,上架,11369,云南雪莲果净果18斤±0.5斤B级（细果）,云南雪莲果净果18斤±0.5斤B级（细果）,云南雪莲果净果18斤±0.5斤B级（细果）,https://azure.summerfarm.net/picture-path/uj43...,999.0,NaN,44240,POP杭州鲜果,9.0,10.0,净重17.5-18斤/B级/细果,"规格:净重17.5-18斤,果规:细果,级别:B级","品种:雪莲果,产地:云南,储藏温度:5-15度",小明


### 获取标果的爬虫数据（当天）

In [146]:
from odps_client import get_odps_sql_result_as_df

all_fruit_category=['千禧樱桃小番茄','红富士苹果','水蜜桃','25号小蜜','冬枣','更多桃','麒麟西瓜','更多梨','更多苹果','更多李']
all_fruit_category.extend(['柠檬','果篮','国产油桃','山竹','巨峰葡萄','阳光玫瑰','进口橙','凯特芒','国产红心火龙果','更多凤梨','更多提子'])
all_fruit_category.extend(['进口红心火龙果','更多蜜瓜','蜂糖李','青脆李','更多柚子','水仙芒','玉菇甜瓜','百香果','更多榴莲','木瓜/杨桃','椰青'])
all_fruit_category.extend(['金枕榴莲','人参果/释迦果','更多龙眼','黄桃','红肉菠萝蜜','三红蜜柚','台芒','吊干杏','无籽红提','更多葡萄','网纹蜜瓜','蜜桔'])
all_fruit_category.extend(['赣南脐橙','黑布林','夏黑葡萄','普通红提','更多柑','蓝莓','西州蜜瓜','更多芒果','水果黄瓜','火腿肠','牛油果','硒砂瓜'])
all_fruit_category.extend(['莲雾/芭乐','其他桔','圆红/血橙','更多橙','白心火龙果','皇冠梨','红布林','进口车厘子','更多荔枝','更多莓','桂味','特小凤','蛇果'])
all_fruit_category.extend(['贡梨','贵妃芒','更多西瓜','白心蜜柚','秋月梨','绿心猕猴桃','羊角蜜','雪莲果/马蹄果','黄元帅苹果','冷冻畜禽食品','更多枣','水产品'])
all_fruit_category.extend(['火参果','红心蜜柚','红毛丹','脆柿','软籽石榴','青芒','鲜山楂','鹰嘴芒'])

biaoguo_df=get_odps_sql_result_as_df(f"""
SELECT  categoryname,backcategoryname,id,competitor,skucode,spider_fetch_time,goodspropdetaillist,createtime
        ,standardprice,finalstandardprice,lasttimestandardprice,finalunitpricecatty,monthsale,attachurlr AS url
        ,unitpricecatty,unit,sellername,grossweight,netweight,specification,babyname,goodsname,goodstype
FROM    (
            SELECT  *
                    ,RANK() OVER (PARTITION BY id,skucode ORDER BY spider_fetch_time DESC ) AS rnk
            FROM    summerfarm_ds.spider_biaoguo_with_prop_product_result_df
            WHERE   ds = MAX_PT('summerfarm_ds.spider_biaoguo_with_prop_product_result_df')
            AND     competitor = '标果-杭州' 
                                     AND categoryname like '%/%'
            -- AND     split_part(categoryname,'/',2) in ('{"','".join(all_fruit_category)}')
        ) 
WHERE   rnk = 1
LIMIT   100000;
""")

print(f"标果的商品数:{len(biaoguo_df)}")
biaoguo_df.head(1)

2024-07-19 15:02:15 - INFO - sql:

SELECT  categoryname,backcategoryname,id,competitor,skucode,spider_fetch_time,goodspropdetaillist,createtime
        ,standardprice,finalstandardprice,lasttimestandardprice,finalunitpricecatty,monthsale,attachurlr AS url
        ,unitpricecatty,unit,sellername,grossweight,netweight,specification,babyname,goodsname,goodstype
FROM    (
            SELECT  *
                    ,RANK() OVER (PARTITION BY id,skucode ORDER BY spider_fetch_time DESC ) AS rnk
            FROM    summerfarm_ds.spider_biaoguo_with_prop_product_result_df
            WHERE   ds = MAX_PT('summerfarm_ds.spider_biaoguo_with_prop_product_result_df')
            AND     competitor = '标果-杭州' 
                                     AND categoryname like '%/%'
            -- AND     split_part(categoryname,'/',2) in ('千禧樱桃小番茄','红富士苹果','水蜜桃','25号小蜜','冬枣','更多桃','麒麟西瓜','更多梨','更多苹果','更多李','柠檬','果篮','国产油桃','山竹','巨峰葡萄','阳光玫瑰','进口橙','凯特芒','国产红心火龙果','更多凤梨','更多提子','进口红心火龙果','更多蜜瓜','蜂糖李','青脆李','更多柚

标果的商品数:3059


,categoryname,backcategoryname,id,competitor,skucode,spider_fetch_time,goodspropdetaillist,createtime,standardprice,finalstandardprice,...,url,unitpricecatty,unit,sellername,grossweight,netweight,specification,babyname,goodsname,goodstype
0,火龙果/进口红心火龙果,进口红心火龙果,236955,标果-杭州,8798126391712,2024-07-19 08:10:31,"[{'产地': '越南'}, {'等级': 'B级'}, {'单果大小': '大果'}, {...",2023-03-04 15:57:01,92.04,92.04,...,https://ugc-pro.biaoguoworks.com/miniprogramWa...,nan,PIECE,公孙策,31.5,27.0,约31.5斤,越南红心火龙果大果20-24头31.5斤B级,越南红心火龙果大果20-24头31.5斤B级,ORDINARY


In [147]:
print(f"标果的所有类目: \n{biaoguo_df['categoryname'].unique()}")

category_list_that_xianmu_wont_sale = set(
    [
        "即食食品/方便罐头",
        "即食食品/海苔食品",
        "饼干糕点/饼干",
        "方便食品/方便速食" "糖果/巧克力/果冻布丁/果冻/布丁",
        "刀具工具/刀具工具",
        "包装配饰/包装配饰",
        "水果盒/带盖盒",
        "陈列道具/陈列道具" "水果盒/托盒",
        "保鲜膜/保鲜膜",
        "标签贴纸/标签贴纸",
        "糖果/巧克力/果冻布丁/龟苓膏/果膏" "即食食品/肉干肉铺",
        "坚果炒货/坚果",
        "其他标品/营养保健",
        "糖果/巧克力/果冻布丁/其他糖果",
        "饼干糕点/散装食品" "酒水/饮料/冲调",
        "即食食品/其他即食",
        "酒水/饮料/酒类",
        "饼干糕点/糕点",
        "标果奶品/标果奶品" "即食食品/蔬菜干/豆干",
        "糖果蜜饯/果干蜜饯",
        "即食食品/火腿肠",
        "水果袋/水果袋" "氛围牌/氛围牌",
        "设计定制/设计定制",
        "水果盒/瓶罐",
        "粽子/米类/面类/粽子/米类/面类",
        "洗护用品/洗护用品",
        "鲜花/鲜花",
        "水产/水产品",
        "即食食品/卤味即食",
    ]
)

print(f"这些类目被过滤掉了:{category_list_that_xianmu_wont_sale}")

# Fix the filtering by using 'isin' method

biaoguo_df = biaoguo_df[
    ~biaoguo_df["categoryname"].isin(category_list_that_xianmu_wont_sale)
].copy()

# Get the length of the filtered DataFrame
print(f"过滤后的标果商品个数:{len(biaoguo_df)}")

标果的所有类目: 
['火龙果/进口红心火龙果' '菠萝/凤梨/更多凤梨' '即食食品/方便罐头' '即食食品/海苔食品' '饼干糕点/饼干' '方便食品/方便速食'
 '糖果/巧克力/果冻布丁/果冻/布丁' '刀具工具/刀具工具' '包装配饰/包装配饰' '水果盒/带盖盒' '陈列道具/陈列道具'
 '水果盒/托盒' '保鲜膜/保鲜膜' '标签贴纸/标签贴纸' '火龙果/白心火龙果' '橙/进口橙' '糖果/巧克力/果冻布丁/龟苓膏/果膏'
 '即食食品/肉干肉铺' '坚果炒货/坚果' '其他标品/营养保健' '糖果/巧克力/果冻布丁/其他糖果' '饼干糕点/散装食品'
 '酒水/饮料/冲调' '即食食品/其他即食' '酒水/饮料/酒类' '山竹/山竹' '饼干糕点/糕点' '标果奶品/标果奶品'
 '菠萝蜜/红肉菠萝蜜' '小颗粒浆果/蓝莓' '即食食品/蔬菜干/豆干' '椰子/椰青' '榴莲/更多榴莲' '柠檬/百香果/百香果'
 '柠檬/百香果/柠檬' '糖果蜜饯/果干蜜饯' '即食食品/火腿肠' '稀奇水果/牛油果' '稀奇水果/莲雾/芭乐' '水果袋/水果袋'
 '氛围牌/氛围牌' '火龙果/国产红心火龙果' '设计定制/设计定制' '稀奇水果/木瓜/杨桃' '芒果/台芒' '苹果/红富士苹果'
 '香瓜/甜瓜/玉菇甜瓜' '苹果/蛇果' '葡萄/阳光玫瑰' '水果盒/瓶罐' '小番茄/千禧樱桃小番茄' '稀奇水果/水果黄瓜' '梨/皇冠梨'
 '粽子/米类/面类/粽子/米类/面类' '柚/更多柚子' '苹果/更多苹果' '提子/更多提子' '桃/水蜜桃' '桃/国产油桃'
 '西瓜/麒麟西瓜' '洗护用品/洗护用品' '蜜瓜/网纹蜜瓜' '鲜花/鲜花' '西瓜/更多西瓜' '蜜瓜/更多蜜瓜' '榴莲/金枕榴莲'
 '蜜瓜/25号小蜜' '水产/水产品' '橙/更多橙' '葡萄/夏黑葡萄' '稀奇水果/人参果/释迦果' '香瓜/甜瓜/羊角蜜' '桃/更多桃'
 '枣/冬枣' '桃/黄桃' '小颗粒浆果/更多莓' '李/蜂糖李' '芒果/水仙芒' '蜜瓜/西州蜜瓜' '芒果/凯特芒' '即食食品/卤味即食'
 '山楂/鲜山楂' '梨/更多梨' '李/青脆李' '柚/白心蜜柚' '李/更多李' '荔枝/桂味' '杏/吊干杏' '柑/更多柑'
 '芒果/贵妃芒

## Azure客户端定义

In [148]:
import os
from openai import AzureOpenAI
import httpx

client = AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint="https://xm-ai.openai.azure.com",
    api_key=os.getenv("AZURE_API_KEY_XM",'please set:AZURE_API_KEY_XM'),
    http_client=httpx.Client(proxies={"http://": None, "https://": None}),
)

## 使用embedding模型做标题的匹配

In [149]:
import sqlite3
import os

# Ensure the directory exists
os.makedirs(os.path.expanduser('~/sqlite'), exist_ok=True)
# Path to the database
db_path = os.path.expanduser('~/sqlite/embeddings.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS embeddings (
    input_text TEXT PRIMARY KEY,
    embedding TEXT
)
"""
)


def get_embedding_directly_from_azure(input: str):
    embbed = client.embeddings.create(model="text-embedding-ada-002", input=input)
    return embbed.to_dict().get("data", [{}])[0].get("embedding")


def get_embedding(input_text):

    # Check if input text already exists in the database
    cursor.execute(
        "SELECT embedding FROM embeddings WHERE input_text = ?", (input_text,)
    )
    result = cursor.fetchone()

    if result:
        embedding = result[0]
        print(f"Found, return the embedding of input_text:{input_text}, {embedding[:50]}")
    else:
        print(f"Not found, call the OpenAI API to get the embedding:{input_text}")
        embedding = str(get_embedding_directly_from_azure(input_text))

        # Insert the new input text and embedding into the database
        cursor.execute(
            "INSERT INTO embeddings (input_text, embedding) VALUES (?, ?)",
            (input_text, embedding),
        )
        conn.commit()
    return embedding


# Example usage
input_text = "你好"
embedding = get_embedding(input_text)
print(json.loads(embedding))

Found, return the embedding of input_text:你好, [0.0001522831298643723, -0.006324205081909895, -0.
[0.0001522831298643723, -0.006324205081909895, -0.0024191036354750395, -0.029225686565041542, -0.04372446611523628, 0.013890438713133335, -0.022407203912734985, -0.008523101918399334, -0.015246530994772911, -0.0193401537835598, 0.035638608038425446, 0.0015762986149638891, 0.004765333142131567, -0.0024697985500097275, -0.008352006785571575, -0.016627971082925797, 0.03421914577484131, -0.015031076967716217, 0.01816149428486824, -0.020151274278759956, -0.00016139238141477108, 0.0013655974762514234, 0.009321548976004124, 0.000830923265311867, -0.0076232654973864555, -0.008016152307391167, 0.010069300420582294, -0.018237538635730743, 0.007211368065327406, -0.008535776287317276, 0.013155360706150532, 0.010779031552374363, -0.029859373345971107, -0.002766047604382038, 0.012078091502189636, -0.013408835977315903, -0.012420282699167728, -0.00189155840780586, 0.020810309797525406, 0.00486038625240325

In [150]:
def get_xianmu_embedding(row: pd.Series):
    sku_name = row["sku_name"]
    if sku_name is None or len(f"{sku_name}") <= 2:
        sku_name = row["pd_name"]
    weight = row["weight"]
    input_text = f"{sku_name}"
    return get_embedding(input_text=input_text)


def get_biaoguo_embedding(row: pd.Series):
    goodsname = row["goodsname"]
    specification = row["specification"]
    return get_embedding(input_text=f"{goodsname}")


def get_category_embedding(category: str):
    return get_embedding(input_text=category)


biaoguo_df["sku_embeddings"] = biaoguo_df.apply(get_biaoguo_embedding, axis=1)
sku_list_df["sku_embeddings"] = sku_list_df.apply(get_xianmu_embedding, axis=1)

biaoguo_df["category_embeddings"] = biaoguo_df['categoryname'].apply(get_category_embedding)
sku_list_df["category_embeddings"] = sku_list_df['categoryName'].apply(get_category_embedding)

Found, return the embedding of input_text:越南红心火龙果大果20-24头31.5斤B级, [-0.004063948057591915, -0.01977352797985077, 0.00
Found, return the embedding of input_text:越南红心火龙果大果20-24头31.5斤A级, [2.3401869839290157e-05, -0.017237156629562378, 0.
Found, return the embedding of input_text:越南红心火龙果大果20-24头31.5斤C级, [0.0040097166784107685, -0.016066236421465874, 0.0
Found, return the embedding of input_text:佳农牌菲律宾凤梨7-8头26斤B级, [-0.015063145197927952, -0.006831612437963486, -0.
Not found, call the OpenAI API to get the embedding:花椒锅巴1*16包*276g（椒香味）


2024-07-19 15:02:15 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2024-07-19 15:02:16 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2024-07-19 15:02:16 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Not found, call the OpenAI API to get the embedding:蒟蒻果冻（百香果味+荔枝味）1*16包*500g
Not found, call the OpenAI API to get the embedding:有骨八角叶浅绿20片中号（32cm）
Not found, call the OpenAI API to get the embedding:4斤装2817透明托盘100个装西瓜葡萄提子


2024-07-19 15:02:16 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2024-07-19 15:02:16 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Not found, call the OpenAI API to get the embedding:2013泡沫托盘200个
Not found, call the OpenAI API to get the embedding:黑色水果垫防滑垫加厚透气1卷1.2m*25m


...

Found, return the embedding of input_text:小番茄/千禧樱桃小番茄, [-0.0064062136225402355, -0.016041351482272148, 0.
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.008319968357682228, -0.0016234922222793102, -0
Found, return the embedding of input_text:方便食品/方便速食, [-0.00831

2024-07-19 15:04:21 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2024-07-19 15:04:21 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2024-07-19 15:04:21 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Found, return the embedding of input_text:稀奇水果/雪莲果/马蹄果, [-0.01090342178940773, -0.007955256849527359, -0.0
Found, return the embedding of input_text:葡萄/更多葡萄, [0.0027884040027856827, -0.020092712715268135, 0.0
Found, return the embedding of input_text:葡萄/更多葡萄, [0.0027884040027856827, -0.020092712715268135, 0.0
Found, return the embedding of input_text:芒果/台芒, [0.002405753592029214, -0.02646847814321518, 0.010
Found, return the embedding of input_text:葡萄/更多葡萄, [0.0027884040027856827, -0.020092712715268135, 0.0
Found, return the embedding of input_text:荔枝/更多荔枝, [-0.014136269688606262, -0.00859029870480299, -0.0
Found, return the embedding of input_text:葡萄/巨峰葡萄, [0.006842796690762043, -0.022290876135230064, 0.00
Found, return the embedding of input_text:蜜瓜/25号小蜜, [-0.011080476455390453, -0.02245950885117054, 0.01
Found, return the embedding of input_text:苹果/黄元帅苹果, [-0.001323237782344222, -0.025461697950959206, -0.
Found, return the embedding of input_text:葡萄/更多葡萄, [0.0027884040027856827, -0

2024-07-19 15:04:22 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2024-07-19 15:04:22 - INFO - HTTP Request: POST https://xm-ai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


Found, return the embedding of input_text:其他鲜果/释迦/莲雾, [0.00508844293653965, -0.016436893492937088, 0.004
Not found, call the OpenAI API to get the embedding:其他鲜果/雪莲果
Found, return the embedding of input_text:其他鲜果/雪莲果, [-0.005764797795563936, -0.025454062968492508, -0.
Found, return the embedding of input_text:凤梨/菠萝/进口凤梨, [0.0077056484296917915, -0.01618051342666149, 0.00
Found, return the embedding of input_text:凤梨/菠萝/进口凤梨, [0.0077056484296917915, -0.01618051342666149, 0.00
Found, return the embedding of input_text:凤梨/菠萝/进口凤梨, [0.0077056484296917915, -0.01618051342666149, 0.00
Found, return the embedding of input_text:凤梨/菠萝/进口凤梨, [0.0077056484296917915, -0.01618051342666149, 0.00
Found, return the embedding of input_text:小番茄/千禧番茄, [-0.0035726295318454504, -0.011677435599267483, 0.
Found, return the embedding of input_text:小番茄/千禧番茄, [-0.0035726295318454504, -0.011677435599267483, 0.
Found, return the embedding of input_text:山竹/山竹, [-0.0027916657272726297, -0.012659567408263683, 0.
Found

In [151]:
import numpy as np
import traceback
from scipy.spatial.distance import cosine

top_n = 20


def calculate_cosine_similarity(embedding1, embedding2):
    if "class 'str'" in f"{type(embedding1)}":
        embedding1 = json.loads(embedding1)
    if "class 'str'" in f"{type(embedding2)}":
        embedding2 = json.loads(embedding2)
    # print(len(embedding1), len(embedding2), type(embedding2))
    return 1 - cosine(embedding1, embedding2)


def find_top_matches_for_xianmu(sku_row: pd.Series):
    # print(f"finding top match rows for:{sku_row['sku_name']}")
    similarities = []
    for index, biaoguo_row in biaoguo_df.iterrows():
        # print(f"matching: {biaoguo_row['goodsname']}")
        similarity_score = calculate_cosine_similarity(
            sku_row["sku_embeddings"], biaoguo_row["sku_embeddings"]
        )
        category_similarity_score = calculate_cosine_similarity(
            sku_row["category_embeddings"], biaoguo_row["category_embeddings"]
        )
        # 类目embedding + sku embedding分数
        similarities.append(
            (biaoguo_row.to_dict(), similarity_score + category_similarity_score)
        )

    # Sort the results based on similarity score in descending order
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Get the top N matches
    top_n_matches = [item[0] for item in sorted_similarities[:top_n]]
    print(
        f"Matched rows for:{sku_row['sku_name']}, {[biaoguo.get('goodsname') for biaoguo in top_n_matches]}"
    )
    return top_n_matches


sku_list_df["top_matches"] = sku_list_df.apply(find_top_matches_for_xianmu, axis=1)
sku_list_df.head(10)[["top_matches", "sku_name", "weight", "price"]]

Matched rows for:海南玫瑰释迦12头净重3.8斤起A级, ['海南黑金刚莲雾1.8斤B级', '海南黑金刚莲雾1.2斤B级', '越南干尧榴莲3头17斤A级', '水果码头菲律宾普通凤梨12头25斤B级', '菲律宾带冠凤梨7-8头26斤A级', '云南紫皮百香果中果3.8斤A级（12对）', '小树牌菲律宾带冠凤梨12头26斤B级', 'SW菲律宾带冠凤梨7-8头25斤A级', '越南干尧榴莲4头16.5斤A级', '云南阳光玫瑰8.6斤B级（3串）', '云南苹果芒大果12头11斤B级', '新西兰红玫瑰苹果135头37斤B级', '新西兰红玫瑰苹果90头36斤B级', 'SW菲律宾带冠凤梨9-10头25斤B级', '云南水仙芒特大果12头15斤B级（4号色）', '浙江阳光玫瑰10斤B级（3串）', '浙江嘉兴阳光玫瑰8.5斤B级（3串）', '菲律宾佳农凤梨5-6头26斤B级', '佳农菲律宾凤梨7-8头25斤B级', '荣盛牌新疆库尔勒香梨大果13.5斤A级']
Matched rows for:云南雪莲果净果18斤±0.5斤B级（细果）, ['云南雪莲果中果18.5斤B级', '云南雪莲果小果4.5斤B级', '云南热农芒小果22斤B级', '云南黄热农芒大果12头15斤B级', '云南苹果芒大果12头11斤B级', '云南水仙芒大果18斤B级（4号色）', '云南热农芒中果12头10斤B级', '云南黄热农芒中果12头11斤B级', '云南丁香蜂糖梨中果10.5斤B级', '云南紫皮百香果中果3.4斤B级（12对）', '云南紫皮百香果大果5斤B级', '云南蜂糖梨3.8斤B级', '云南大青芒中果10-12头18斤B级', '云南水仙芒中果15斤B级（4号色）', '云南水仙芒特大果19斤B级（4号色）', '云南蜂糖梨中果10.5斤C级', '海南青金桔中果2.4斤B级（5合1）', '云南紫皮百香果中果3.8斤A级（12对）', '云南水仙芒特大果12头15斤B级（5号色）', '云南石林人参果中果9.5斤B级（八合一）']
Matched rows for:云南雪莲果净果4斤±0.3斤B级（细果）, ['云南雪莲果小果4.5斤B级', '云南雪莲果中果18.5斤B级', '云南紫皮百香果中果3.4斤B级（12对）', '云南黄热

,top_matches,sku_name,weight,price
41,"[{'categoryname': '稀奇水果/莲雾/芭乐', 'backcategoryn...",海南玫瑰释迦12头净重3.8斤起A级,净重3.8-4.2斤/A级/12头,999.0
184,"[{'categoryname': '稀奇水果/雪莲果/马蹄果', 'backcategor...",云南雪莲果净果18斤±0.5斤B级（细果）,净重17.5-18斤/B级/细果,999.0
183,"[{'categoryname': '稀奇水果/雪莲果/马蹄果', 'backcategor...",云南雪莲果净果4斤±0.3斤B级（细果）,净重3.7-4.3斤/B级/小果,999.0
130,"[{'categoryname': '菠萝/凤梨/更多凤梨', 'backcategoryn...",SW凤梨12kgA级9头,净重23-24斤/A级/单果9头,999.0
53,"[{'categoryname': '菠萝/凤梨/更多凤梨', 'backcategoryn...",佳农凤梨25-27斤A级7-8头,净重25-27斤/A级/7-8头,999.0
32,"[{'categoryname': '菠萝/凤梨/更多凤梨', 'backcategoryn...",佳农菲律宾黑钻凤梨7-8头B级净重23斤+,净重23-25斤/B级/7-8头,999.0
133,"[{'categoryname': '菠萝/凤梨/更多凤梨', 'backcategoryn...",都乐有冠凤梨11kgA级8头,净重18.5-20斤/A级/单果8头,999.0
155,"[{'categoryname': '小番茄/千禧樱桃小番茄', 'backcategory...",山东玲珑小番茄2.5±0.2斤B级,毛重2.3-2.6斤/B级/单果7g,999.0
154,"[{'categoryname': '小番茄/千禧樱桃小番茄', 'backcategory...",山东玲珑小番茄6斤B级,毛重5.8-7斤/B级/单果7g,999.0
185,"[{'categoryname': '山竹/山竹', 'backcategoryname':...",泰国2A山竹/净果15斤/B级（油竹）,净重15-15斤/B级/2A,999.0


### 写入到HTML文件

In [152]:
from IPython.core.display import HTML

css = """
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.0.0/dist/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
<style type=\"text/css\">
table {
    color: #333;
    font-family: unset;
    font-size: 12px;
    line-height: 1.5;
    width: 1024px;
    border-collapse:
    collapse; 
    border-spacing: 0;
    font-family: "SF Pro SC", "SF Pro Text", "SF Pro Icons", "PingFang SC", "Helvetica Neue", "Helvetica", "Arial", sans-serif;
}

tr{
    border-bottom: 1px solid #C1C3D1;
}

tr:nth-child(even) {
    background-color: #F8F8F8;
}

td, th {
    /* border: 1px solid transparent; No more visible border */
    height: 30px;
}

th {
    background-color: #DFDFDF; /* Darken header a bit */
    font-weight: bolder;
    min-width: 100px;
    text-align: center;
}

td {
    /* background-color: #FAFAFA;
    text-align: center; */
}

ol li{
    text-align: left;
}
.biaoguo-container {
    display:flex;
}
.no-list-style ul{
    padding-left: 0.2vw;
}
.no-list-style li{
    list-style: none;
    font-size: smaller;
}
.no-list-style li.sku-title{
    font-weight:bolder;
}
.xianmu-container{
    /* border-right: 1px solid lightblue; */
}
</style>
"""


def display_xianmu_html(row):
    img = row["picture"]
    unit_price = round(row["price"] / row.get("weightNum", 0) / 2, 2)
    content = f"""<div class="xianmu-container no-list-style">
    <img width="80" src="{img}">
    <ul>
    <li class="sku-title">{row['sku']}, {row["sku_name"]}</li>
    <li>鲜沐售价: ¥{row["price"]}</li>
    <li>单价(斤,毛重): ¥{unit_price}</li>
    <li>提报价: ¥{row['supplyPrice']}</li>
    <li>类目:{row.get('categoryName','')}</li>
    <li>规格:{row["weight"]}</li>
    <li>毛重:{row.get('weightNum', 0)*2}斤, 净重:{row.get('netWeightNum', 0)*2}斤</li>
    <li>买手:{row["buyer_name"]}</li>
    </ul></div>"""
    return content.replace("\n", "")


display_count = 10


def display_matched_biaoguo_html(row):
    top_matches = row["top_matches"]
    contents_of_single_item = []
    for index, item in enumerate(top_matches):
        if index >= display_count:
            # print(f"超过了配置数量:{display_count},跳过")
            break
        img = item["url"]
        single_item = f"""<div class="biaoguo-goods-item no-list-style">
        <img width="80" src="{img}">
        <ul>
        <li class="sku-title">{item['skucode']}, {item['goodsname']}, {item['specification']}</li>
        <li>标果价格: ¥{item['finalstandardprice']}</li>
        <li>单价(斤,毛重): ¥{item['finalunitpricecatty']}</li>
        <li>类目:{item['categoryname']}</li>
        <li>毛重:{item['grossweight']}斤, 净重:{item['netweight']}斤</li>
        </ul></div>"""
        contents_of_single_item.append(single_item)

    return f"""<div class="biaoguo-container">{''.join(contents_of_single_item)}</div>""".replace(
        "\n", ""
    )


sku_list_df["鲜沐POP商品"] = sku_list_df.apply(display_xianmu_html, axis=1)
sku_list_df["标果商品(Top10)"] = sku_list_df.apply(display_matched_biaoguo_html, axis=1)

date_of_now = datetime.now().strftime("%Y-%m-%d")
html_content = css + sku_list_df[["鲜沐POP商品", "标果商品(Top10)"]].to_html(
    escape=False, index=False, classes="table dataframe"
)
html_content = f'<html><head><meta charset="UTF-8"><meta name="title" content="POP商品和标果商品的比较-{date_of_now}"></head><body>{html_content}</body></html>'
# display(HTML(html_content))

file_path = f"./data/pop/鲜沐POP和标果-杭州的比较-{date_of_now}.html"

# 保存HTML到本地文件：
with open(file_path, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"写入HTML成功！{file_path}")

写入HTML成功！./data/pop/鲜沐POP和标果-杭州的比较-2024-07-19.html


### 写入CSV，供大家参考

In [158]:
sku_list_for_csv = []
for _, row in sku_list_df.iterrows():
    csv_object = {}
    unit_price = round(row["price"] / row.get("weightNum", 0) / 2, 2)
    xianmu_content = f"""{row['sku']}, {row["sku_name"]}
鲜沐售价: ¥{row["price"]}, 单价(斤,毛重): ¥{unit_price}
类目:{row.get('categoryName','')}
规格:{row["weight"]}
毛重:{row.get('weightNum', 0)*2}斤, 净重:{row.get('netWeightNum', 0)*2}斤"""
    csv_object["顺鹿达SKU"] = xianmu_content

    top_matches = row["top_matches"]
    for i in range(0, 20):
        if i >= len(top_matches):
            break
        item = top_matches[i]
        single_item = f"""{item['skucode']}, {item['goodsname']}, {item['specification']}
标果价格: ¥{item['finalstandardprice']}
单价(斤,毛重): ¥{item['finalunitpricecatty']}
类目:{item['categoryname']}
毛重:{item['grossweight']}斤, 净重:{item['netweight']}斤"""
        csv_object[f"标果SKU{i+1}"] = single_item
    sku_list_for_csv.append(csv_object)

sku_list_for_csv_df = pd.DataFrame(sku_list_for_csv)
sku_list_for_csv_df.to_csv(
    f"./data/pop/顺鹿达SKU_vs_标果SKU_top20_{date_of_now}.csv", index=False
)
sku_list_for_csv_df.head(2)

,顺鹿达SKU,标果SKU1,标果SKU2,标果SKU3,标果SKU4,标果SKU5,标果SKU6,标果SKU7,标果SKU8,标果SKU9,...,标果SKU11,标果SKU12,标果SKU13,标果SKU14,标果SKU15,标果SKU16,标果SKU17,标果SKU18,标果SKU19,标果SKU20
0,"1296235406315, 海南玫瑰释迦12头净重3.8斤起A级\n鲜沐售价: ¥999....","2862903484335, 海南黑金刚莲雾1.8斤B级, 约1.8斤\n标果价格: ¥32...","9175886670141, 海南黑金刚莲雾1.2斤B级, 约1.2斤\n标果价格: ¥20...","7335910004878, 越南干尧榴莲3头17斤A级, 约17斤\n标果价格: ¥261...","8863572844505, 水果码头菲律宾普通凤梨12头25斤B级, 约25斤\n标果价格...","4700690865846, 菲律宾带冠凤梨7-8头26斤A级, 约26斤\n标果价格: ¥...","3338581348220, 云南紫皮百香果中果3.8斤A级（12对）, 约3.8斤\n标果...","2353440541380, 小树牌菲律宾带冠凤梨12头26斤B级, 约26斤\n标果价格:...","3779493306589, SW菲律宾带冠凤梨7-8头25斤A级, 约25斤\n标果价格:...","3729143904313, 越南干尧榴莲4头16.5斤A级, 约16.5斤\n标果价格: ...",...,"5085162620080, 云南苹果芒大果12头11斤B级, 约11斤\n标果价格: ¥6...","7314807079683, 新西兰红玫瑰苹果135头37斤B级, 约37斤\n标果价格: ...","2803323582422, 新西兰红玫瑰苹果90头36斤B级, 约36斤\n标果价格: ¥...","3110842017304, SW菲律宾带冠凤梨9-10头25斤B级, 约25斤\n标果价格...","3631095147969, 云南水仙芒特大果12头15斤B级（4号色）, 约15斤\n标果...","5052671684577, 浙江阳光玫瑰10斤B级（3串）, 约10斤\n标果价格: ¥9...","3996156423560, 浙江嘉兴阳光玫瑰8.5斤B级（3串）, 约8.5斤\n标果价格...","8632773405678, 菲律宾佳农凤梨5-6头26斤B级, 约26斤\n标果价格: ¥...","9450997317791, 佳农菲律宾凤梨7-8头25斤B级, 约25斤\n标果价格: ¥...","8488418235158, 荣盛牌新疆库尔勒香梨大果13.5斤A级, 约13.5斤\n标果..."
1,"1298087783322, 云南雪莲果净果18斤±0.5斤B级（细果）\n鲜沐售价: ¥9...","8135243118371, 云南雪莲果中果18.5斤B级, 约18.5斤\n标果价格: ¥...","5621854962006, 云南雪莲果小果4.5斤B级, 约4.5斤\n标果价格: ¥24...","5742057925281, 云南热农芒小果22斤B级, 约22斤\n标果价格: ¥114....","9228190722202, 云南黄热农芒大果12头15斤B级, 约15斤\n标果价格: ¥...","5085162620080, 云南苹果芒大果12头11斤B级, 约11斤\n标果价格: ¥6...","9595441828364, 云南水仙芒大果18斤B级（4号色）, 约18斤\n标果价格: ...","5254329563477, 云南热农芒中果12头10斤B级, 约10斤\n标果价格: ¥5...","9733924266567, 云南黄热农芒中果12头11斤B级, 约11斤\n标果价格: ¥...","3444078913777, 云南丁香蜂糖梨中果10.5斤B级, 约10.5斤\n标果价格:...",...,"3645633796138, 云南紫皮百香果大果5斤B级, 约5斤\n标果价格: ¥69.4...","5617466163971, 云南蜂糖梨3.8斤B级, 约3.8斤\n标果价格: ¥16.9...","4291232129102, 云南大青芒中果10-12头18斤B级, 约18斤\n标果价格:...","4594397626805, 云南水仙芒中果15斤B级（4号色）, 约15斤\n标果价格: ...","7658429891229, 云南水仙芒特大果19斤B级（4号色）, 约19斤\n标果价格:...","1489595029263, 云南蜂糖梨中果10.5斤C级, 约10.5斤\n标果价格: ¥...","3998000994956, 海南青金桔中果2.4斤B级（5合1）, 约2.4斤\n标果价格...","3338581348220, 云南紫皮百香果中果3.8斤A级（12对）, 约3.8斤\n标果...","2029230736866, 云南水仙芒特大果12头15斤B级（5号色）, 约15斤\n标果...","2073134559836, 云南石林人参果中果9.5斤B级（八合一）, 约9.5斤\n标果..."


## 使用语言模型（GPT3）做匹配

In [153]:
model = "gpt-35-turbo-16k"
# model = "gpt-4o"

llm_top_n = 3


def get_top_matched_biaoguo_sku(xianmu_sku_specification: str, biaoguo_sku_to_match=[]):
    sku_to_match = "\n".join(biaoguo_sku_to_match)
    messages = [
        {
            "role": "user",
            "content": f"以下商品中，和这个商品：“{xianmu_sku_specification}”最相似的前{llm_top_n}个是？请从商品名字、规格、包装净重这三个方面综合考虑。**请你直接返回你认为最相似的{llm_top_n}个商品，每行一个商品，不要返回其他任何信息**：\n{sku_to_match}",
        },
    ]
    completion = client.chat.completions.create(
        model=model, temperature=0.7, max_tokens=4095, messages=messages
    )

    response = completion.choices[0].message.content
    if (
        len(completion.choices) <= 0
        or f"{completion.choices[0].finish_reason}" == "content_filter"
    ):
        print(f"azure过滤了本次请求:{completion.choices[0].to_dict()}")
    if response is None:
        print(f"azure API返回了异常:{completion.to_dict()}")

    print(f"xianmu_title:{xianmu_sku_specification}\nresponse:{response}")
    return response


# get_top_matched_biaoguo_sku(xianmu_sku_specification=specification_test, biaoguo_sku_to_match=biaoguo_sku_to_match)

### 调用LLM进行匹配

- 太费事了，先这样吧

In [154]:
def get_top_matched_by_llm(xianmu_sku):
    specification = f"skucode:{xianmu_sku['sku']}, 品名:{xianmu_sku['categoryName']}-{xianmu_sku['sku_name']}"
    specification = f"{specification}, 规格:{xianmu_sku['weight']}, 毛重:{xianmu_sku.get('weightNum', 0)*2}斤, 净重:{xianmu_sku.get('netWeightNum', 0)*2}斤"
    top_matches = xianmu_sku["top_matches"]
    biaoguo_sku_to_match = []
    for biaoguo_matched in top_matches:
        sku = f"skucode:{biaoguo_matched['skucode']}, 品名:{biaoguo_matched['categoryname']}-{biaoguo_matched['goodsname']}"
        sku = f"{sku}, 规格:{biaoguo_matched['specification']}, 毛重:{biaoguo_matched['grossweight']}斤, 净重:{biaoguo_matched['netweight']}斤"
        print(sku)
        biaoguo_sku_to_match.append(sku)
    return get_top_matched_biaoguo_sku(
        xianmu_sku_specification=specification,
        biaoguo_sku_to_match=biaoguo_sku_to_match,
    )

# LLM太费事了，先这样吧
# sku_list_df["top_matched_gpt35"] = sku_list_df.apply(get_top_matched_by_llm, axis=1)

## 把鲜沐的SKU写入到本地CSV

In [155]:
sku_list_clean_df = sku_list_df[
    [
        "sku",
        "pdId",
        "categoryName",
        "openSale",
        "sku_name",
        "pd_name",
        "price",
        "supplyPrice",
        "areaName",
        "netWeightNum",
        "weightNum",
        "weight",
        "sku_spec",
        "spu_spec",
        "buyer_name",
        "picture",
    ]
].copy()
sku_list_clean_df.rename(
    columns={
        "categoryName": "类目",
        "openSale": "是否上架",
        "areaName": "运营区域",
        "netWeightNum": "净重(公斤)",
        "weightNum": "毛重(公斤)",
        "picture": "图片链接",
        "weight": "SKU规格",
        "price": "售价",
        "supplyPrice": "供应商提报价",
        "buyer_name": "买手名字",
    },
    inplace=True,
)
sku_list_clean_df["供应商提报价"].fillna(0, inplace=True)
sku_list_clean_df.sort_values(by=["类目", "pd_name"], inplace=True)
sku_list_clean_df.to_csv(f"./data/pop/鲜果POP全部SKU_{date_of_now}.csv", index=False)
sku_list_clean_df.head(10)

,sku,pdId,类目,是否上架,sku_name,pd_name,售价,供应商提报价,运营区域,净重(公斤),毛重(公斤),SKU规格,sku_spec,spu_spec,买手名字,图片链接
41,1296235406315,11153,其他鲜果/释迦/莲雾,上架,海南玫瑰释迦12头净重3.8斤起A级,海南玫瑰释迦12头净重3.8斤起A级,999.0,0.0,POP杭州鲜果,1.9,2.50,净重3.8-4.2斤/A级/12头,"规格:净重3.8-4.2斤,果规:12头,级别:A级","品种:玫瑰,产地:海南,储藏温度:5-15度",小明,https://azure.summerfarm.net/picture-path/hbw9...
184,1298087783322,11369,其他鲜果/雪莲果,上架,云南雪莲果净果18斤±0.5斤B级（细果）,云南雪莲果净果18斤±0.5斤B级（细果）,999.0,0.0,POP杭州鲜果,9.0,10.00,净重17.5-18斤/B级/细果,"规格:净重17.5-18斤,果规:细果,级别:B级","品种:雪莲果,产地:云南,储藏温度:5-15度",小明,https://azure.summerfarm.net/picture-path/uj43...
183,1298828206681,11368,其他鲜果/雪莲果,上架,云南雪莲果净果4斤±0.3斤B级（细果）,云南雪莲果净果4斤±0.3斤B级（细果）,999.0,0.0,POP杭州鲜果,2.0,2.30,净重3.7-4.3斤/B级/小果,"规格:净重3.7-4.3斤,果规:小果,级别:B级","品种:雪莲果,产地:云南,储藏温度:5-15度",小明,https://azure.summerfarm.net/picture-path/6rma...
130,1262274344480,11303,凤梨/菠萝/进口凤梨,上架,SW凤梨12kgA级9头,SW凤梨12kgA级9头,999.0,0.0,POP杭州鲜果,12.0,13.00,净重23-24斤/A级/单果9头,"规格:净重23-24斤,果规:单果9头,级别:A级","品种:凤梨,产地:菲律宾,储藏温度:5-15度",風,https://azure.summerfarm.net/picture-path/xded...
53,1262403361881,11165,凤梨/菠萝/进口凤梨,上架,佳农凤梨25-27斤A级7-8头,佳农凤梨25-27斤A级7-8头,999.0,0.0,POP杭州鲜果,12.5,14.50,净重25-27斤/A级/7-8头,"规格:净重25-27斤,果规:7-8头,级别:A级","品种:凤梨,产地:菲律宾,储藏温度:5-15度",風,https://azure.summerfarm.net/picture-path/jxnk...
32,1262646388520,11144,凤梨/菠萝/进口凤梨,上架,佳农菲律宾黑钻凤梨7-8头B级净重23斤+,佳农菲律宾黑钻凤梨7-8头B级净重23斤+,999.0,0.0,POP杭州鲜果,11.5,13.00,净重23-25斤/B级/7-8头,"规格:净重23-25斤,果规:7-8头,级别:B级","品种:黑钻,产地:菲律宾,储藏温度:5-15度",小明,https://azure.summerfarm.net/picture-path/tbrq...
133,1262100828645,11306,凤梨/菠萝/进口凤梨,上架,都乐有冠凤梨11kgA级8头,都乐有冠凤梨11kgA级8头,999.0,0.0,POP杭州鲜果,10.0,11.50,净重18.5-20斤/A级/单果8头,"规格:净重18.5-20斤,果规:单果8头,级别:A级","品种:凤梨,产地:菲律宾,储藏温度:5-15度",風,https://azure.summerfarm.net/picture-path/fzcz...
155,1217710254618,11334,小番茄/千禧番茄,上架,山东玲珑小番茄2.5±0.2斤B级,山东玲珑小番茄2.5±0.2斤B级,999.0,0.0,POP杭州鲜果,1.0,1.25,毛重2.3-2.6斤/B级/单果7g,"规格:毛重2.3-2.6斤,果规:单果7g,级别:B级","品种:千禧,产地:山东,储藏温度:5-15度",大表哥,https://azure.summerfarm.net/picture-path/vnyr...
154,1217456718585,11333,小番茄/千禧番茄,上架,山东玲珑小番茄6斤B级,山东玲珑小番茄6斤B级,999.0,0.0,POP杭州鲜果,3.0,3.50,毛重5.8-7斤/B级/单果7g,"规格:毛重5.8-7斤,果规:单果7g,级别:B级","品种:千禧,产地:山东,储藏温度:5-15度",大表哥,https://azure.summerfarm.net/picture-path/wrdo...
185,1231185578243,11370,山竹/山竹,上架,泰国2A山竹/净果15斤/B级（油竹）,泰国2A山竹/净果15斤/B级（油竹）,999.0,0.0,POP杭州鲜果,7.5,8.00,净重15-15斤/B级/2A,"规格:净重15-15斤,果规:2A,级别:B级","品种:油竹,产地:泰国,储藏温度:5-15度",小明,https://azure.summerfarm.net/picture-path/99ur...
